## Model training and deployment to Azure

#### Installation of accessory libraries

In [ ]:
import warnings
warnings.simplefilter("ignore")
import pycaret 

print(pycaret.__version__)

In [ ]:
# !pip install azure-storage-blob

In [ ]:
# !pip install git+https://github.com/amjadraza/pycaret.git@feature/gcp_azure_np_docs

### Model Development

In [ ]:
from pycaret.datasets import get_data
dataset = get_data('diamond')

In [ ]:
data = dataset.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
from pycaret.regression import *
exp_reg101 = setup(data = data, target = 'Price', session_id=123, use_gpu = True, verbose = False)

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
tuned_lightgbm = tune_model(lightgbm)

In [ ]:
predict_model(tuned_lightgbm );

### Finalize the model

In [ ]:
final_lightgbm  = finalize_model(tuned_lightgbm )

In [ ]:
print(final_lightgbm)

In [ ]:
predict_model(final_lightgbm);

# Deploy the model to Microsoft Azure

This is the code to implement the model in Microsoft Azure using the `pycaret` features.

In [ ]:
def create_container(container_name):

  # Create the container
    container_client = blob_service_client.create_container(container_name)

    return container_client

def upload_blob(container_name, source_file_name, destination_blob_name):

  # Create a blob client using the local file name as the name for the blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=destination_blob_name)

    print("\nUploading to Azure Storage as blob:\n\t" + source_file_name)

  # Upload the created file
    with open(source_file_name, "rb") as data:
        blob_client.upload_blob(data)

def download_blob(container_name, source_blob_name, destination_file_name):
  # Download the blob to a local file
    print("\nDownloading blob to \n\t" + destination_file_name)

  # Create a blob client using the local file name as the name for the blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=source_blob_name)

    if destination_file_name is not None: 
        with open(destination_file_name, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

        print(
            "Blob {} downloaded to {}.".format(
                source_blob_name, destination_file_name
            )
        )

In [ ]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

#### Here you should put the access token (Connection String)

In [ ]:
## Enter connection string when running in google colab
connect_str = '' 
print(connect_str)

In [ ]:
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

##### Create the container in the Blob Storage

In [ ]:
# Create a unique name for the container
container_name = "machinelearninginsurance"
container_client = create_container(container_name)
model_dir = 'modelo'

In [ ]:
# Save Model Local and upload to Azure
model_name_azure = 'lightgbm-reg-101'
save_model(final_lightgbm, 
           model_name= model_name_azure, 
           verbose=False)

model_src = model_name_azure +'.pkl'
model_dst = str(model_name_azure)+'.pkl'

##### Upload the model

In [ ]:
upload_blob(container_name, model_src, model_dst)